In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [6]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={"class":'a-price a-text-price a-size-medium apexPriceToPay'}).find("span", attrs={"class":"a-offscreen"}).string.strip()

    except AttributeError:
        price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [7]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=xbox+consoles&rh=n%3A720022&pf_rd_i=23508887011&pf_rd_m=ATVPDKIKX0DER&pf_rd_p=434db2ed-6d53-4c59-b173-e8cd550a2e4f&pf_rd_r=SBW4AVVPASZH1B48TC0J&pf_rd_s=merchandised-search-5&pf_rd_t=101&ref=nb_sb_noss"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [8]:
amazon_df

,title,price,rating,reviews,availability
0,Xbox Series X – Forza Horizon 5 Bundle,$536.64,4.7 out of 5 stars,433 ratings,Only 1 left in stock - order soon
1,"Microsoft FMQ-00042 Console, 18.5"" x 12"" x 5"",...",$349.99,4.5 out of 5 stars,5 ratings,Only 1 left in stock - order soon
2,Xbox Series S - Holiday Console,,4.8 out of 5 stars,"2,597 ratings",Not Available
3,XBOX Console,,4.2 out of 5 stars,"1,062 ratings",Only 1 left in stock - order soon
4,Xbox Series S 512GB All Digital Holiday Console,,4.4 out of 5 stars,2 ratings,Not Available
5,Microsoft Xbox Series S Console - 512GB (Renewed),$239.99,4.5 out of 5 stars,848 ratings,Not Available
6,Microsoft Original Xbox Console with Controlle...,$182.62,3.8 out of 5 stars,472 ratings,Only 2 left in stock - order soon
7,Xbox One X 1TB Console With Wireless Controller,$379.97,4.5 out of 5 stars,116 ratings,Only 1 left in stock - order soon
8,Xbox 360 Core Console Video Game System,$123.12,4.7 out of 5 stars,156 ratings,Only 16 left in stock - order soon
9,Microsoft Xbox Console,$149.94,3.8 out of 5 stars,397 ratings,Only 1 left in stock - order soon
